In [ ]:
!pip install git+https://github.com/keepsimpler/zero

In [ ]:
from zero import *

In [ ]:
species = [50,50,50,50]
connectance = [0.05]*4
Gs = [after_DAG(niche(s, c)[1]) for s,c in zip(species, connectance)]
ns = [16,16,16,16,16]
model = ComplexNet(Gs, ns, xception)


In [ ]:
num_params(model)

In [ ]:
def get_data(size, woof, bs, workers=None):
    if   size<=128: path = URLs.IMAGEWOOF_160 if woof else URLs.IMAGENETTE_160
    elif size<=224: path = URLs.IMAGEWOOF_320 if woof else URLs.IMAGENETTE_320
    else          : path = URLs.IMAGEWOOF     if woof else URLs.IMAGENETTE
    path = untar_data(path)

    n_gpus = num_distrib() or 1
    if workers is None: workers = min(8, num_cpus()//n_gpus)

    return (ImageList.from_folder(path).split_by_folder(valid='val')
            .label_from_folder().transform(([flip_lr(p=0.5)], []), size=size)
            .databunch(bs=bs, num_workers=workers)
            .presize(size, scale=(0.35,1))
            .normalize(imagenet_stats))

In [ ]:
data = get_data(size=128, woof=True, bs=64)

In [ ]:
learn = Learner(data, model, metrics=accuracy).to_fp16()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True) 

In [ ]:
for _ in range(3):
    Gs = [after_DAG(niche(s, c)[1]) for s,c in zip(species, connectance)]
    model = ComplexNet(Gs, ns, xception)
    learn = Learner(data, model, metrics=accuracy).to_fp16()
    learn.fit_one_cycle(30, 1e-2) 